In [1]:
import streamlit as st
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain. prompts import PromptTemplate
from langchain. chains import LLMChain
from langchain_community.document_loaders import CSVLoader

# pip install streamlit, langchain, langchain-community, python-dotenv, faiss-cpu
# https://python.langchain.com/docs/tutorials/rag/

In [2]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_anthropic import ChatAnthropic
from langchain_google_genai import ChatGoogleGenerativeAI

import os
from dotenv import load_dotenv


C:\Users\otimi\anaconda3\envs\python\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
load_dotenv(dotenv_path='.env')

# https://platform.openai.com/api-keys
openai_api_key = os.getenv("OPENAI_API_KEY")

# https://console.groq.com/keys
groq_api_key = os.getenv("GROQ_API_KEY")

# https://console.anthropic.com/dashboard
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")

# https://aistudio.google.com/app/apikey
gemini_api_key = os.getenv("GEMINI_API_KEY")

In [5]:
# https://www.planalto.gov.br/ccivil_03/LEIS/2002/L10406compilada.htm

#loader = CSVLoader(file_path="output.csv", encoding='utf-8')
loader = CSVLoader(file_path="dados/lei.txt", encoding='utf-8')
documents = loader.load()
documents

[Document(metadata={'source': 'dados/lei.txt', 'row': 0}, page_content='Presidência da República: Casa Civil'),
 Document(metadata={'source': 'dados/lei.txt', 'row': 1}, page_content='Presidência da República: Subchefia para Assuntos Jurídicos'),
 Document(metadata={'source': 'dados/lei.txt', 'row': 2}, page_content='Presidência da República: LEI Nº 10.406\nNone: DE 10 DE JANEIRO DE 2002'),
 Document(metadata={'source': 'dados/lei.txt', 'row': 3}, page_content='Presidência da República: ÍNDICE'),
 Document(metadata={'source': 'dados/lei.txt', 'row': 4}, page_content='Presidência da República: Vigência'),
 Document(metadata={'source': 'dados/lei.txt', 'row': 5}, page_content='Presidência da República: Lei de Introdução às normas do Direito Brasileiro'),
 Document(metadata={'source': 'dados/lei.txt', 'row': 6}, page_content='Presidência da República: (Vide Lei nº 14.195\nNone: de 2021)'),
 Document(metadata={'source': 'dados/lei.txt', 'row': 7}, page_content='Presidência da República: Vi

In [7]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
# pip install faiss-cpu

embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
db = FAISS.from_documents(documents, embeddings)

In [8]:
question = "Onde é o domicílio da pessoa natural ?"
similar_response = db.similarity_search(question,k=3)
best_practice = [doc.page_content for doc in similar_response]
best_practice

['None: Art. 70. O domicílio da pessoa natural é o lugar onde ela estabelece a sua residência com ânimo definitivo.',
 'None: Art. 71. Se,porém,a pessoa natural tiver diversas residências,onde,alternadamente,viva,considerar-se-á domicílio seu qualquer delas.',
 'None: Art. 72. É também domicílio da pessoa natural,quanto às relações concernentes à profissão,o lugar onde esta é exercida.']

In [9]:
template = """
Você é um assistente virtual de um escritório de advocacia.
Sua função será responder e-mails que recebemos de clientes respondendo perguntas sobre a lei do Novo Código Civil
O novo Código Civil é estabelecido pela LEI Nº 10.406, de 10 de janeiro de 2002 e os trechos relevantes da lei são dados por {best_practice}.

Siga todas as regras abaixo:
1. Você deve buscar se comportar de maneira cordial e solícita.
2. Suas respostas devem ser bem similares ou até identicas aos termos da lei.
3. Alguns dos e-mails podem conter links e informações irrelevantes. Preste atenção apenas no conteúdo útil da mensagem.
Aqui está uma mensagem da dúvida recebida de um cliente.
{message}

Escreva a melhor resposta que eu deveria enviar para solucionar a dúvida apresentada pela pergunta deste cliente.
Encerre com a seguinte assinatura:

Equipe Fale Conosco 
email de contato: faleconosco@email.com.br
"""

In [10]:
from langchain.prompts.prompt import PromptTemplate
prompt = PromptTemplate(input_variables=['message','best_practice'],template=template)

In [11]:
llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key=openai_api_key)
chain = prompt | llm
response = chain.invoke({"message":question,"best_practice":best_practice})
print(response.content)


Prezado(a) cliente,

O domicílio da pessoa natural é definido pelo Art. 70 do Novo Código Civil, que estabelece que é o lugar onde ela estabelece a sua residência com ânimo definitivo. Caso a pessoa natural tenha diversas residências, onde viva alternadamente, considerar-se-á domicílio qualquer uma delas, conforme disposto no Art. 71. Além disso, em relação às relações concernentes à profissão, o lugar onde a atividade é exercida também é considerado domicílio, conforme o Art. 72.

Estamos à disposição para quaisquer outras dúvidas.

Atenciosamente,

Equipe Fale Conosco  
email de contato: faleconosco@email.com.br
